# Setup and Basic Text Generation

In [1]:
%load_ext autoreload
%autoreload 2

# Import necessary modules
import asyncio
from dotenv import load_dotenv
import os
from shuscribe.services.llm.session import LLMSession
from shuscribe.services.llm.providers.provider import (
    Message, MessageRole, GenerationConfig, TextContent
)

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
ANTHROPIC_API_KEY = os.environ["ANTHROPIC_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

DEFAULT_ANTHROPIC_MODEL = "claude-3-5-sonnet-20241022"
DEFAULT_GEMINI_MODEL = "gemini-2.0-flash-001"
DEFAULT_OPENAI_MODEL = "gpt-4o-mini"
DEFAULT_THINKING_MODEL = "o3-mini-2025-01-31"

# Helper function to run async code in notebook
async def run_async(coro):
    return await coro

# Initialize session and simple messages
async def initialize():
    async with LLMSession.session_scope() as session:
        # Create simple conversation
        messages = [
            Message(role=MessageRole.SYSTEM, content="You are a helpful assistant that speaks in a concise manner."),
            Message(role=MessageRole.USER, content="What is the capital of France?")
        ]
        return session, messages

session, messages = await run_async(initialize())

## Basic Text Generation with Different Providers

In [2]:
# # Using OpenAI
# async def test_openai():
#     async with LLMSession.session_scope() as session:
#         provider = await session.get_provider("openai")
#         response = await provider.generate(
#             messages=messages,
#             model=DEFAULT_OPENAI_MODEL,
#             config=GenerationConfig(temperature=0.7)
#         )
#         return response.text

# openai_response = await run_async(test_openai())
# print(f"OpenAI response: {openai_response}")

In [3]:
# # Using Anthropic
# async def test_anthropic():
#     async with LLMSession.session_scope() as session:
#         response = await session.generate(
#             messages=messages,
#             provider="anthropic",
#             model=DEFAULT_ANTHROPIC_MODEL,
#             config=GenerationConfig(temperature=0.7)
#         )
#         return response.text

# anthropic_response = await run_async(test_anthropic())
# print(f"Anthropic response: {anthropic_response}")

In [4]:
# Using Gemini
async def test_gemini():
    async with LLMSession.session_scope() as session:
        response = await session.generate(
            messages=messages,
            provider="gemini",
            model=DEFAULT_GEMINI_MODEL,
            config=GenerationConfig(temperature=0.7)
        )
        return response.text

gemini_response = await run_async(test_gemini())
print(f"Gemini response: {gemini_response}")

Gemini response: Paris.



In [14]:
# Streaming response example
async def test_streaming():
    async with LLMSession.session_scope() as session:
        # Create a streaming config
        config = GenerationConfig(temperature=0.7, stream=True)
        
        print("Streaming response:")
        
        async for chunk in session.generate_stream(
            messages=[Message(role=MessageRole.USER, content="Can you create a short 3 paragraph story about a cat?")],
            provider="anthropic",
            model= DEFAULT_ANTHROPIC_MODEL,
            config=config
        ):
            # Print each chunk as it arrives
            if isinstance(chunk, str):
                print(chunk, end="", flush=True)
            else:
                # If it's an LLMResponse object
                if chunk.text:
                    print(chunk.text, end="", flush=True)
        
        print("\nStreaming complete!")

await run_async(test_streaming())

Streaming response:


ERROR:root:Error in Anthropic generate_stream: 'TextBlock' object has no attribute 'get'
Traceback (most recent call last):
  File "/Users/jimmyyao/gitrepos/shuscribe/backend/shuscribe/services/llm/providers/anthropic_provider.py", line 377, in generate_stream
    if chunk.content_block.get('type') == 'text':
       ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jimmyyao/gitrepos/shuscribe/backend/.venv/lib/python3.12/site-packages/pydantic/main.py", line 891, in __getattr__
    raise AttributeError(f'{type(self).__name__!r} object has no attribute {item!r}')
AttributeError: 'TextBlock' object has no attribute 'get'



Error: 'TextBlock' object has no attribute 'get'
Streaming complete!


In [7]:
# Define a structured output model
from pydantic import BaseModel, Field
from typing import List

class Capital(BaseModel):
    name: str
    country: str
    population: int
    landmarks: List[str]

# Query with structured output
async def test_structured_output():
    structured_messages = [
        Message(role=MessageRole.SYSTEM, content="You provide factual information."),
        Message(role=MessageRole.USER, content="Provide information about Paris as the capital of France.")
    ]
    
    async with LLMSession.session_scope() as session:
        provider = await session.get_provider("openai")
        
        # Parse the response into a Capital object
        response = await provider.parse(
            messages=structured_messages,
            model=DEFAULT_OPENAI_MODEL,
            response_format=Capital
        )
        
        if response.parsed_response:
            return response.parsed_response
        else:
            return f"Failed to parse: {response.text}"

structured_result = await run_async(test_structured_output())
print(f"Structured result: {structured_result}")
print(f"Capital name: {structured_result['name']}")
print(f"Landmarks: {', '.join(structured_result['landmarks'])}")

Structured result: {'name': 'Paris', 'country': 'France', 'population': 2140526, 'landmarks': ['Eiffel Tower', 'Louvre Museum', 'Notre-Dame Cathedral', 'Sacré-Cœur Basilica', 'Champs-Élysées', 'Arc de Triomphe', 'Palace of Versailles', 'Montmartre']}
Capital name: Paris
Landmarks: Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, Sacré-Cœur Basilica, Champs-Élysées, Arc de Triomphe, Palace of Versailles, Montmartre


# Session Management and Provider Reuse

In [10]:
# Test session management
async def test_session_reuse():
    results = []
    
    # Get the singleton instance
    session = await LLMSession.get_instance()
    
    # Use the same provider instance for multiple requests
    provider = await session.get_provider("openai")
    
    for i in range(3):
        response = await provider.generate(
            messages=[
                Message(role=MessageRole.USER, content=f"Count to {i+1} briefly.")
            ],
            model=DEFAULT_OPENAI_MODEL
        )
        results.append(response.text)
    
    return results

session_results = await run_async(test_session_reuse())
for i, result in enumerate(session_results):
    print(f"Query {i+1}: {result}")

Query 1: 1
Query 2: 1, 2.
Query 3: 1, 2, 3.


# 9. Testing Provider Capabilities


In [ ]:
# Check provider capabilities
async def test_capabilities():
    capabilities = {}
    
    async with LLMSession.session_scope() as session:
        for provider_name in ["openai", "anthropic", "gemini"]:
            provider = await session.get_provider(provider_name)
            
            capabilities[provider_name] = {
                "structured_output": provider.supports_structured_output,
                "multimodal_input": provider.supports_multimodal_input,
                "tool_use": provider.supports_tool_use,
                "parallel_tool_calls": provider.supports_parallel_tool_calls,
                "search": provider.supports_search,
                "extended_thinking": provider.supports_extended_thinking,
                "citations": provider.supports_citations
            }
    
    return capabilities

provider_capabilities = await run_async(test_capabilities())
provider_capabilities

{'openai': {'structured_output': True,
  'multimodal_input': True,
  'tool_use': True,
  'parallel_tool_calls': True,
  'search': True,
  'extended_thinking': False,
  'citations': False},
 'anthropic': {'structured_output': True,
  'multimodal_input': True,
  'tool_use': True,
  'parallel_tool_calls': True,
  'search': False,
  'extended_thinking': True,
  'citations': True},
 'gemini': {'structured_output': True,
  'multimodal_input': True,
  'tool_use': True,
  'parallel_tool_calls': True,
  'search': True,
  'extended_thinking': False,
  'citations': False}}